# Asset Tracker Tool - Tracking serial numbers among multiple data sets

## Overview

- This tool compares data exports in CVS formate. Examples of datasets that can be compared include:

 - Installed Base
 - PageSmart (FMAudit data)
 - Project Rollout Schedule(s)
 - Monthly Billing File
 
- Functions:
 - Phase 1: [current] Compare databases 1-to-1 to identify missing devices.
 - Phase 2: Data accuracy 
 - Phase 3: Predictive corrections

## Identify Data Files and Global Variables 

In [31]:
import unicodecsv

#####################################
#               Formatting for comparison ease of read
#####################################
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

#####################################
#               Convert Excel to csv (.xlsx - data must be on Sheet1)
#####################################
import xlrd
import csv

def csv_from_excel(file1, new_file_name):
    wb = xlrd.open_workbook(file1)
    sh = wb.sheet_by_name('Sheet1')
    your_csv_file = open(new_file_name, 'w')
    wr = csv.writer(your_csv_file, quoting=csv.QUOTE_ALL)

    for rownum in range(sh.nrows):
        wr.writerow(sh.row_values(rownum))

    your_csv_file.close()

# runs the csv_from_excel function:
#csv_from_excel('MGM IB.xlsx', "MGM IB.csv")

#####################################
#       WHICH FILES ARE WE LOOKING AT?
#####################################
'''
    src files are standard exports, typically <1mb
    src_large_n files are typically larger than a 1mb, up to 15mb
    #IMPORTANT: Ensure gspmps.com instead of https://tabs.toshibameters.com to ensure you’re looking at the most recent record when multiple FMAudit servers are installed for a single customer
    #IMPORTANT: this is a single file for all ROS's 
'''
src1 = 'MGM IB by 4 customer numbers 8.30.3019.csv'
src2 = 'MGM FMA 8.15.2019 Device Change Worksheet.csv'
src3 = ''
src4 = ''
# src3 = 'MGM Project ROS Summary.csv'
# src4 = 'MGM Billing 8.27.2019.csv'
src5 = ''
src_large_1 = 'Full IB 8.21.2019 - serials only.csv'
# src_large_2 = 'Full IB from 2016-2019 BRIEF.csv'
src_large_2 = ''

# source_file_names = [
#     'MGM IB by 4 customer numbers 8.30.3019.csv',
#     'MGM FMA 8.15.2019 Device Change Worksheet.csv',
#     'MGM Project ROS Summary.csv',
#     'EINVOICE-T0BUBPS-30-SEP-19.csv'
#     ]

# source_file_names_large = [
#     'Full IB 8.21.2019 - serials only.csv',
#     'Full IB from 2016-2019 BRIEF.csv'
#     ]

source_file_names = [
    'AEG Project Implementation Roll Out Schedule.csv',
    'Copy of AUGUST 2019 WELLS FARGO - ANSCHUTZ ENTERTAINMENT INVOICE.csv'
    ]
print("Number of entries in source_file_names: ", len(source_file_names))

source_file_names_large = []
print("Number of entries in source_file_names_large: ", len(source_file_names_large))



#####################################
#       MISC information
#####################################
#Customer Reference Number for Oracle Installed Base
cust_numbers = list()
cust_numbers = ("T0BS49A", "T0BUQW1", "T0BV7VF", "T0BVMSY")
print("Number of entries in cust_numbers: ", len(cust_numbers))

serial_number_name_variants = list()
serial_number_name_variants = ['SERIAL_NUMBER','Serial Number','Toshiba Serial Number','SerialNbr', '\ufeffSERIAL_NUMBER']
print("Number of entries in serial_number_name_variants: ", len(serial_number_name_variants))

key_and_value_to_filter_out = list()
key_and_value_to_filter_out.append(('CUSTOMER_LOCATION_STATE','MD'))
key_and_value_to_filter_out.append(('CUSTOMER_NAME', 'MGM STUDIOS'))
key_and_value_to_filter_out.append(('CUSTOMER_NAME', 'MGM GALLERIES LLC'))
key_and_value_to_filter_out.append(('CUSTOMER_NAME', 'MGM LIQUOR WAREHOUSE'))
key_and_value_to_filter_out.append(('CUSTOMER_NAME', 'MGM MORTGAGE'))

print("Number of entries in key_and_value_to_filter_out: ", len(key_and_value_to_filter_out))

#What are some headings you'd like to see if we find missing assets?
potential_data_headers_keys = list()
potential_data_headers_keys = ['serial_number', \
                               'ACTION', 'Completed Install Date', 'Last Report Date' , \
                               'Primary', 'STATUS for Delivery / Installation', \
                               'Location Name', 'Completed Install Date' \
                               'Ship To / Company Name','Address1', 'Model', \
                               'Ship-To Name', 'Ship To Address1', 'Install Date', 'Status'\
                               'INSTALL_DATE','CUSTOMER_NAME','CUSTOMER_LOCATION_ADDRESS_1']     
print("Number of entries in potential_data_headers_keys: ", len(potential_data_headers_keys))

print("I did the thing!")

Number of entries in source_file_names:  2
Number of entries in source_file_names_large:  0
Number of entries in cust_numbers:  4
Number of entries in serial_number_name_variants:  5
Number of entries in key_and_value_to_filter_out:  5
Number of entries in potential_data_headers_keys:  16
I did the thing!


## Load Data from CSVs

In [32]:
#####################################
#       IMPORT SCRIPTS FOR EACH THAT EXPORTS SAMPLE OF SERIAL NUMBER
#####################################


def import_cvs_into_list(source_cvs):
    """
        Imports CSV into list
        Args:
            source_cvs
        Returns:
            data_list
    """
    #creates a list where each row is list entry. Each list entry is a collection of Dict's
    data_list = []
    with open(source_cvs, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        for row in reader:
            data_list.append(row)
    #List value of row:
    print(color.UNDERLINE + "Sample pulled from " + source_cvs + color.END + ", entry from 0th list entry: ")
    print(data_list[0])
    return data_list

ib_devices = list()
pagesmart_devices = list()
ros_devices = list()
ib_full_devices = list()
billing_devices = list()
ib_full_brief_devices = list()

if src1:
    ib_devices = import_cvs_into_list(src1)
if src2:
    pagesmart_devices = import_cvs_into_list(src2)
if src3:
    ros_devices = import_cvs_into_list(src3)
if src_large_1:
    ib_full_devices_serials = import_cvs_into_list(src_large_1)
if src4:
    billing_devices = import_cvs_into_list(src4)
if src5:
    misc_devices = import_cvs_into_list(src5)
if src_large_2:
    ib_full_brief_devices = import_cvs_into_list(src_large_2)

#####################################
#       IMPORT SCRIPTS FOR EACH THAT EXPORTS SAMPLE OF SERIAL NUMBER [VERION 2.0]
#####################################
print("----------------------------------------------------------")
source_file_lists=list()
source_file_lists_large=list()

for num, file_name in enumerate(source_file_names, start=1):
    print("\nImporting {}: {} as new list: data_set_{}".format(num, file_name, num))
    locals()["data_set_" + str(num)] = import_cvs_into_list(file_name)
    source_file_lists.append(locals()["data_set_" + str(num)])

for num, file_name in enumerate(source_file_names_large, start=1):
    print("\nImporting: {} as new list: data_set_large_{}".format(file_name, num))
    locals()["data_set_large_" + str(num)] = import_cvs_into_list(file_name)
    source_file_lists_large.append(locals()["data_set_large_" + str(num)])

# print(len(source_file_lists[0]))
# print(source_file_lists[0][0])

Sample pulled from MGM IB by 4 customer numbers 8.30.3019.csv, entry from 0th list entry: 
OrderedDict([('\ufeffPRODUCT_NUMBER', '63451262'), ('INSTALL_DATE', '4/1/2019'), ('GOLD_FLAG', 'Y'), ('CUSTOMER_REFERENCE', 'T0BS49A'), ('CUSTOMER_NAME', 'MGM RESORTS INTERNATIONAL'), ('SRC_PARTY_SITE_NUMBER', '2601839'), ('CUSTOMER_LOCATION_ADDRESS_1', '71 E HARMON AVE'), ('CUSTOMER_LOCATION_ADDRESS_2', ''), ('CUSTOMER_LOCATION_CITY', 'LAS VEGAS'), ('CUSTOMER_LOCATION_STATE', 'NV'), ('CUSTOMER_LOCATION_ZIP', '89109-4539'), ('SRV_DEALER_REFERENCE', '00740600'), ('SRV_DEALER_NAME', 'TOSHIBA BUSINESS SOLUTIONS AZ-CO'), ('SRV_DEALER_LOCATION_ADDRESS_1', 'C/O MGM RESORTS'), ('SRV_DEALER_LOCATION_ADDRESS_2', '5014 BOND STREET'), ('SRV_DEALER_LOCATION_CITY', 'LAS VEGAS'), ('SRV_DEALER_LOCATION_STATE', 'NV'), ('SRV_DEALER_LOCATION_ZIP', '89118-1575'), ('ORIG_DEALER_REFERENCE', '00743600'), ('ORIG_DEALER_NAME', 'TABS NO COMP'), ('MODEL_NUMBER', 'ESTUDIO2802AF'), ('SERIAL_NUMBER', 'SCUBJ51625'), ('EXPIRED

## Harmonize data keys e.g. Serial Number columns

In [33]:
#####################################
#               Harmonize Keys      #
#####################################

def harmonize_serial_number_key(list_a):
    """
    Since Serial Number is primary key, they key name sshould be the same among all tables
    ArgS:
        list_a
    Returns:
        none
    """
    for row in list_a:
        if ('serial_number') in row:
            continue
        for sn_variant in serial_number_name_variants:
            if sn_variant in row:
                row['serial_number'] = row[sn_variant]
                del row[sn_variant]
        if ('serial_number') not in row:
            print("Issue: Did not find serial number key.")

            
if src1:
    harmonize_serial_number_key(ib_devices)
    print(ib_devices[0]['serial_number'])
    
if src2:
    harmonize_serial_number_key(pagesmart_devices)
    print(pagesmart_devices[0]['serial_number'])

if src3:
    harmonize_serial_number_key(ros_devices)
    print(ros_devices[0]['serial_number'])

#NOTE: src_large_1 is already a list of just serial numbers to reduce resource drain on computer
# if src_large_1:
#     harmonize_serial_number_key(ib_full_devices_serials)
#     print(ib_full_devices_serials[0]['serial_number'])    
    
if src4:
    harmonize_serial_number_key(billing_devices)
    print(billing_devices[0]['serial_number'])

if src_large_2:
    harmonize_serial_number_key(ib_full_brief_devices)
    print(ib_full_brief_devices[0]['serial_number'])

print("I did the thing!")

print("----------------------------------------------------------")
for data_set in source_file_lists:
    harmonize_serial_number_key(data_set)
    print(data_set[0]['serial_number'])

for data_set in source_file_lists_large:
    harmonize_serial_number_key(data_set)
    print(data_set[0]['serial_number'])

SCUBJ51625
U63478D3J153685
I did the thing!
----------------------------------------------------------
SCNCJ36223
SC1EJ15306


## Fixing Data Types - FUNCTION

In [34]:
#####################################
#     Add data types other than string, IB portion and functions
#####################################

from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '' or date == None:
        return None
    else:
        return dt.strptime(date, '%m/%d/%Y')
        #return dt.strptime(date, '%m/%d/%Y %H:%M:%S')
        
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)
    
print("I did the thing!")

I did the thing!


## Fixing Data Types

In [35]:
if src1:    
    # Clean up the data types an INSTALL BASE table
    for ib_device in ib_devices:
        ib_device['INSTALL_DATE'] = parse_date(ib_device['INSTALL_DATE'])
        #ib_device['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
        ib_device['GOLD_FLAG'] = ib_device['GOLD_FLAG'] == 'True'

print("I did the thing!")
    
print("----------------------------------------------------------")
for data_set in source_file_lists:
    if 'INSTALL_DATE' in data_set[0].keys():
        for data_entry in data_set:
            data_entry['INSTALL_DATE'] = parse_date(data_entry['INSTALL_DATE'])
    if 'GOLD_FLAG' in data_set[0].keys():
        for data_entry in data_set:
            data_entry['GOLD_FLAG'] = data_entry['GOLD_FLAG'] == 'True'
    if 'Last Report Date' in data_set[0].keys():
        for data_entry in data_set:
            data_entry['Last Report Date'] = parse_date(data_entry['Last Report Date'])

print("I did the thing!")

I did the thing!
----------------------------------------------------------
I did the thing!


## Find/List Duplicate Serial Numbers

In [36]:
#####################################
#                 Find unique and non-blank, post duplicates
#####################################

## Find the total number of rows and the number of unique students (account keys)
## in each table.

def find_unique_and_none_blank(input_list):
    unique_values = set()
    duplicate_values = set()
    blank_counter = 0
    for row in input_list:
        if not row['serial_number']:
            blank_counter += 1
        elif row['serial_number'] not in unique_values:
            unique_values.add(row['serial_number'])
        elif row['serial_number'] is not None:
            duplicate_values.add(row['serial_number'])
            
    
    print('Total devices checked: ' +  str(len(input_list)))
    print('Total unique devices: ' + str(len(unique_values)))
    print('Total blanks: ' + str(blank_counter))
    
    if (len(input_list)!=len(unique_values)):
        print("Total duplicates devices: " + str(len(input_list)-len(unique_values)))
        if(len(duplicate_values) > 1 or duplicate_values):
            print(color.BOLD + "Duplicate records found. Please investigate then delete applicable record(s)." \
                  + color.END)
    return duplicate_values
    
#####################################
#               List orig vs duplicates count
#####################################

print (src1), print(find_unique_and_none_blank(ib_devices), "\n") if src1 else None
print (src2), print(find_unique_and_none_blank(pagesmart_devices), "\n") if src2 else None
print (src3), print(find_unique_and_none_blank(ros_devices), "\n") if src3 else None
print (src4), print(find_unique_and_none_blank(billing_devices), "\n") if src4 else None

print("----------------------------------------------------------")

for list_name, data_set in zip(source_file_names,source_file_lists):
    print (list_name)
    print (find_unique_and_none_blank(data_set), "\n")

    
print("I did the thing!")

MGM IB by 4 customer numbers 8.30.3019.csv
Total devices checked: 1898
Total unique devices: 1897
Total blanks: 0
Total duplicates devices: 1
Duplicate records found. Please investigate then delete applicable record(s).
{'JPCCL9N0FJ'} 

MGM FMA 8.15.2019 Device Change Worksheet.csv
Total devices checked: 4059
Total unique devices: 3494
Total blanks: 565
Total duplicates devices: 565
set() 



----------------------------------------------------------
AEG Project Implementation Roll Out Schedule.csv
Total devices checked: 77
Total unique devices: 50
Total blanks: 27
Total duplicates devices: 27
set() 

Copy of AUGUST 2019 WELLS FARGO - ANSCHUTZ ENTERTAINMENT INVOICE.csv
Total devices checked: 163
Total unique devices: 81
Total blanks: 1
Total duplicates devices: 82
Duplicate records found. Please investigate then delete applicable record(s).
{'S75289200F3ZHX', 'S75289200F3ZH9', 'S75289200F3ZHT', 'SCNEJ51688', 'SCNEJ51680', 'SC1EJ15590', 'SCNEJ51781', 'SCNEJ51941', 'SC1EJ15722', 'SC1EJ15

## Account for "Silly S"

In [37]:
#####################################
#    Compare serial numbers that may have "Silly S"
#####################################
#Some serials have the "Silly S" at the beginning.  Check values with added 'S' and removed first character to check against.
def check_serial_silly_s(input_serial, check_against_list):
    if (input_serial in check_against_list):
        return input_serial
    elif (("S"+input_serial) in check_against_list):
#         print('S added to beginning of Serial for ' + str(input_serial))
        return ("S"+input_serial)
    elif ((input_serial[1:]) in check_against_list):
#         print('First Character removed to beginning of Serial for ' + str(input_serial))
        return (input_serial[1:])
    else:
        return None
    
#test above function
if src1:
    print(check_serial_silly_s("S"+ib_devices[0]['serial_number'],"S" + ib_devices[0]['serial_number']))
    print(check_serial_silly_s("S"+ib_devices[0]['serial_number'],(ib_devices[0]['serial_number'])))
print("I did the thing!")

print("----------------------------------------------------------")

#test above function
print(check_serial_silly_s("S"+source_file_lists[0][0]['serial_number'],"S" + source_file_lists[0][0]['serial_number']))
print(check_serial_silly_s("S"+source_file_lists[0][0]['serial_number'],(source_file_lists[0][0]['serial_number'])))
print("I did the thing!")

SSCUBJ51625
SCUBJ51625
I did the thing!
----------------------------------------------------------
SSCNCJ36223
SCNCJ36223
I did the thing!


## Create unique serials lists (removing duplicates)

In [38]:
#####################################
#     Create unique value list (remove duplicates)
#####################################

## Find any one student ib_devices where the student is missing from the daily engagement table.
## Output that enrollment.

def find_unique_and_return_list(input_list):
    unique_values = set()
    for row in input_list:
        if row['serial_number'] not in unique_values:
            unique_values.add(row['serial_number'])
    print('Total unique devices after check: ' + str(len(unique_values)))
    if (len(input_list)!=len(unique_values)):
        print("This function returned a list that removed the following number of duplicates: " \
              + str(len(input_list)-len(unique_values)) + '\n')
    return unique_values

if src1: 
    print(color.UNDERLINE +'ib_devices total rows: ' + str(len(ib_devices)) + color.END)
    ib_devices_unique_device_serials = find_unique_and_return_list(ib_devices)

if src2:
    print(color.UNDERLINE +'pagesmart_devices total rows: ' + str(len(pagesmart_devices)) + color.END)
    pagesmart_devices_unique_device_serials = find_unique_and_return_list(pagesmart_devices)

if src3:
    print(color.UNDERLINE +'ros_devices total rows: ' + str(len(ros_devices)) + color.END)
    ros_devices_unique_device_serials = find_unique_and_return_list(ros_devices)

if src4:
    print(color.UNDERLINE +'billing_devices total rows: ' + str(len(billing_devices)) + color.END)
    billing_devices_unique_device_serials = find_unique_and_return_list(billing_devices)
    
print("----------------------------------------------------------")

# for list_name, data_set in zip(source_file_names,source_file_lists):
#     print (color.UNDERLINE + list_name + color.END )
#     print (find_unique_and_none_blank(data_set), "\n")
source_file_unique_serials = list()
source_file_unique_serials_large = list()

print(source_file_names)

for num, data_set in enumerate(source_file_lists, start=1):
    print("\nCreating new list of unique serial numbers from file: {}. \nNamed: data_set_unique_serials_{}".format(source_file_names[num-1], num))
    locals()["data_set_unique_serials_" + str(num)] = find_unique_and_return_list(data_set)
    source_file_unique_serials.append(locals()["data_set_unique_serials_" + str(num)])
    
for num, data_set in enumerate(source_file_lists_large, start=1):
    print("\nCreating new list of unique serial numbers from file: {}. \nNamed: data_set_unique_serials_large_{}".format(source_file_names_large[num-1], num))
    locals()["data_set_unique_serials_large_" + str(num)] = find_unique_and_return_list(data_set)
    source_file_unique_serials_large.append(locals()["data_set_unique_serials_large_" + str(num)])
    
print("I did the thing!")


ib_devices total rows: 1898
Total unique devices after check: 1897
This function returned a list that removed the following number of duplicates: 1

pagesmart_devices total rows: 4059
Total unique devices after check: 3495
This function returned a list that removed the following number of duplicates: 564

----------------------------------------------------------
['AEG Project Implementation Roll Out Schedule.csv', 'Copy of AUGUST 2019 WELLS FARGO - ANSCHUTZ ENTERTAINMENT INVOICE.csv']

Creating new list of unique serial numbers from file: AEG Project Implementation Roll Out Schedule.csv. 
Named: data_set_unique_serials_1
Total unique devices after check: 51
This function returned a list that removed the following number of duplicates: 26


Creating new list of unique serial numbers from file: Copy of AUGUST 2019 WELLS FARGO - ANSCHUTZ ENTERTAINMENT INVOICE.csv. 
Named: data_set_unique_serials_2
Total unique devices after check: 82
This function returned a list that removed the followi

## Missing Records - Filter for Search Part 1: FUNCTION

In [39]:
def filter_out(filter_keys_values_list, unique_serials_list, orig_data_set):
    """
    #####################################
        Removes entries based on filter
        Args: 
            list_a 
            key
            key 2 (if applicable)
        Returns:
            filtered_list
    #####################################
    """
    filtered_list = list()
    removed_list = list()
    print('\nBefore: quantity of lists\' entries: ' + str(len(unique_serials_list)))

    # create data list using unique serial numbers:
    full_data_unique = list()
    for unique_entry in unique_serials_list:
        # find orig data entry
        for orig_entry in orig_data_set:
            if (unique_entry == orig_entry['serial_number']):
                full_data_unique.append(orig_entry)
                break
    
    filtered_list =  full_data_unique
    
    for filter_item in filter_keys_values_list:
        key = filter_item[0]
        value = filter_item[1]
        print("Removing key/value: ", key, value)
        if key in full_data_unique[0].keys():
            for entry in full_data_unique:
                if entry[key] == value:
                    #print("Removing: ", entry['serial_number'])
                    removed_list.append(entry)
                    filtered_list.remove(entry)
                    
    if (removed_list):
        print("Number of entries removed: " + color.RED + str(len(removed_list)) + color.END + " out of a total " + str(len(unique_serials_list)))
        
    print('After: quantity of lists\' entries: ' + str(len(full_data_unique)))
    return filtered_list
print("I did the thing!")

I did the thing!


## Missing Records - Filter for Search Part 2: CRITERIA

In [40]:
#What's in IB that should not be in PS/ROS?
ib_data_to_filter = list()
ib_data_to_filter.append(('CUSTOMER_LOCATION_STATE','MD'))
ib_data_to_filter.append(('CUSTOMER_NAME', 'MGM STUDIOS'))
ib_data_to_filter.append(('CUSTOMER_NAME', 'MGM GALLERIES LLC'))
ib_data_to_filter.append(('CUSTOMER_NAME', 'MGM LIQUOR WAREHOUSE'))
ib_data_to_filter.append(('CUSTOMER_NAME', 'MGM MORTGAGE'))

print("Length of ib_data_filterd: ", len(ib_data_to_filter))
ps_data_to_filter_from_ib = list()

if src1 and src3: 
    #What's in PS that would not be in IB?
    ps_data_to_filter_from_ib = list()
    print("Length of ps_data_filterd_from_ib: ", len(ps_data_to_filter_from_ib))

if src2 and src3: 
    #What's in PS that won't be in ROS?
    ps_data_to_filter_from_ros = list()
    print("Length of ps_data_filterd_from_ros: ", len(ps_data_to_filter_from_ros))

if src3 and src2: 
    #Note: All ROS (installed) should be in PS - No filters suggested
    ros_data_to_filter_from_ps = list()
    print("Length of ros_data_to_filter_from_ps: ", len(ros_data_to_filter_from_ps))

if src1 and src3: 
    #Note: All ROS (installed) should be in IB - No filters suggested
    ros_data_to_filter_from_ib = list()
    print("Length of ros_data_to_filter_from_ib: ", len(ros_data_to_filter_from_ib))

if src4: 
    #Note: All BILLING (installed) should be in IB - No filters suggested
    billing_data_to_filter = list()
    print("Length of ros_data_to_filter_from_ib: ", len(billing_data_to_filter))

print("I did the thing!")





Length of ib_data_filterd:  5
I did the thing!


## Missing Records - Filter for Search Part 3: PERFORM

In [41]:
#ros_found_in_ib = check_and_list_missing_serials_in_lists(ros_devices_unique_device_serials, ib_devices_unique_device_serials)


#INSTALL BASE FILTERS
if src1:
    ib_data_filtered = filter_out(ib_data_to_filter, ib_devices_unique_device_serials, ib_devices)

if src2 and src1:
    ps_data_filtered_for_ib = filter_out(ps_data_to_filter_from_ib, pagesmart_devices_unique_device_serials, pagesmart_devices)
# if src2 and src3:
#     ps_data_filtered_for_ros = filter_out(ps_data_to_filter_from_ros, pagesmart_devices_unique_device_serials, pagesmart_devices)
    
# if src3 and src2: 
#     ros_data_filtered_for_ps = filter_out(ros_data_to_filter_from_ps, ros_devices_unique_device_serials, ros_devices)
# if src3 and src1:
#     ros_data_filtered_for_ib = filter_out(ros_data_to_filter_from_ib, ros_devices_unique_device_serials, ros_devices)
    
# if src4 and src1: 
#     billing_data_filtered_for_ib = filter_out(billing_data_to_filter, billing_devices_unique_device_serials, billing_devices)

print("----------------------------------------------------------")
key_and_value_to_filter_out

# for list_name, data_set in zip(source_file_names,source_file_lists):
#     print (color.UNDERLINE + list_name + color.END )
#     print (find_unique_and_none_blank(data_set), "\n")
source_file_unique_filtered_data = list()
source_file_unique_filtered_data_large = list()

for num, data_set in enumerate(source_file_lists, start=1):
    print("\nCreating new list of filtered devices: {}. \nNamed: data_set_unique_filtered_data_{}".format(source_file_names[num-1], num))
    locals()["data_set_unique_filtered_data_" + str(num)] = filter_out(key_and_value_to_filter_out, source_file_unique_serials[num-1], data_set)
    source_file_unique_filtered_data.append(locals()["data_set_unique_filtered_data_" + str(num)])
    
# for num, data_set in enumerate(source_file_lists_large, start=1):
#     print("\nCreating new list of unique serial numbers from file: {}. \nNamed: data_set_unique_serials_large_{}".format(source_file_names_large[num-1], num))
#     locals()["data_set_unique_serials_large_" + str(num)] = find_unique_and_return_list(data_set)
#     source_file_unique_serials_large.append(locals()["data_set_unique_serials_large_" + str(num)])

print("I did the thing!")


Before: quantity of lists' entries: 1897
Removing key/value:  CUSTOMER_LOCATION_STATE MD
Removing key/value:  CUSTOMER_NAME MGM STUDIOS
Removing key/value:  CUSTOMER_NAME MGM GALLERIES LLC
Removing key/value:  CUSTOMER_NAME MGM LIQUOR WAREHOUSE
Removing key/value:  CUSTOMER_NAME MGM MORTGAGE
Number of entries removed: 232 out of a total 1897
After: quantity of lists' entries: 1665

Before: quantity of lists' entries: 3495
After: quantity of lists' entries: 3495
----------------------------------------------------------

Creating new list of filtered devices: AEG Project Implementation Roll Out Schedule.csv. 
Named: data_set_unique_filtered_data_1

Before: quantity of lists' entries: 51
Removing key/value:  CUSTOMER_LOCATION_STATE MD
Removing key/value:  CUSTOMER_NAME MGM STUDIOS
Removing key/value:  CUSTOMER_NAME MGM GALLERIES LLC
Removing key/value:  CUSTOMER_NAME MGM LIQUOR WAREHOUSE
Removing key/value:  CUSTOMER_NAME MGM MORTGAGE
After: quantity of lists' entries: 51

Creating new 

## Missing Records - Find/List Missing Serials From Each DB FUNCTION

In [42]:
from tabulate import tabulate
from prettytable import PrettyTable
from datetime import datetime

#comparison function between two lists 
def check_and_list_missing_serials_in_lists(list_a, list_b):
    """
    #####################################
        Takes in two lists (each list is a list of dicts for a devices e.g. serial_number key to asdf1234 value)
            and compares them to each other finding matches and "not_found" based on serial_number.
        Args: 
            list_a - entries to be check, return values are based on these entries
            list_b - entires to check against
        Returns:
            found_matches - a list of entries from list_a that were found in list_b based on serial_number
            not_found - a list of entries from list_a that were NOT found in list_b based on serial_number
    #####################################
    """
    
    #Print output 
    print(color.UNDERLINE +'\nComparison stats:\n' + color.END + \
     " • " + 'Quantity of primary lists\' entries: ' + str(len(list_a)))

    found_matches = list()
    not_found = list()
    for device in list_a:
        device_serial = device['serial_number'] 
        if check_serial_silly_s(device_serial, list_b):
            found_matches.append(device)
            continue
        else:
            not_found.append(device)
            
    #Print output         
    print(" • " + 'Matches found: ' + str(len(found_matches)))

    if (not_found):
        output_sample_records(not_found)

    return found_matches, not_found

print("I did the thing!")


I did the thing!


## Missing Records - Find/List Missing Serials From Each OUTPUT FUNCTION

In [43]:
def output_sample_records(list_a):
    rows = list()
    headers = list()
    #Print output         
    print (" • " + 'Total Missing: ' + str(len(list_a)) + " \n" \
    + color.BOLD + "Please investigate then add applicable record(s). Some Key fields: \n" + color.END)

    print("----------------------------------------------------------")
    
    for potential_key in potential_data_headers_keys:
        if potential_key in list_a[0].keys():
            headers.append(potential_key)

    #Check if ROS headers exist and then output
    if (headers):
        for entry in list_a:
            #temp new row variable to add to rows as single line
            new_row = list()
            for found_potential_header in headers:
                new_row.append(entry[found_potential_header])
            rows.append(new_row)
                
    #else (no header matches)
    else:
        headers = 'serial_number'
        for entry in list_a:
            rows.append(entry['serial_number'])

    print(tabulate(rows, headers))

print("I did the thing!")

I did the thing!


## Missing Records Part 1 - Find/List Missing Serials

In [44]:
#####################################
#                 LIST MISSING SERIAL NUMBERS
#####################################
# if src3 and src1:
#     print(color.UNDERLINE +'\nList comparison:' + color.END + ' If any devices are listed below, they are in... \n'  + \
#          " • " + "" + color.GREEN + src3 + color.END + " but are... \n" + \
#          " • " +  color.BOLD + color.RED + "Missing from " + src1 + color.END)
#     ros_found_in_ib = check_and_list_missing_serials_in_lists(ib_data_filtered, ps_data_filtered_for_ib)
    
# if src2 and src1:
#     print(color.UNDERLINE +'\nList comparison:' + color.END + ' If any devices are listed below, they are in... \n'  + \
#          " • " + "" + color.GREEN + src2 + color.END + " but are... \n" + \
#          " • " +  color.BOLD + color.RED + "Missing from " + src1 + color.END)
#     ps_found_in_ib = check_and_list_missing_serials_in_lists(ib_data_filtered, pagesmart_devices_unique_device_serials)
    
print("----------------------------------------------------------")

#TO DO: itereate among filtered lists to check against unique serials lists

total_lists = len(source_file_lists)

print("This section will compare each list against each other and provide missing device information.\nThere are {} lists.".format(total_lists))  

if total_lists > 1:
    for num, data_set in enumerate(source_file_unique_filtered_data, start=0):
        compare_to_index = num+1
        while compare_to_index < total_lists:
            print(color.UNDERLINE +'\nList comparison:' + color.END + ' If any devices are listed below, they are in... \n'  + \
            " • " + "" + color.GREEN + source_file_names[num] + color.END + " but are missing from... \n" + \
            " • " +  color.BOLD + color.RED + "Missing from " + source_file_names[compare_to_index] + color.END)
            print("Coding troubleshooting (if needed) note: Iterator index: {}. \ncompare_to_index: {}.".format(num, \
                                                                        compare_to_index))    
            locals()["data_set_matches_" + str(num)] = check_and_list_missing_serials_in_lists(source_file_unique_filtered_data[num], \
                                                                                               source_file_unique_serials[compare_to_index])
            #source_file_unique_filtered_data.append(locals()["data_set_unique_filtered_data_" + str(num)])
            compare_to_index = compare_to_index+1

print("\nI did the thing!")


'''
    TO DO
    Reverse order check e.g. 2 against 1
    Dyanmic or earlier selected additional data fields e.g. "Install Status" vs "Status of delivery/install"
'''



----------------------------------------------------------
This section will compare each list against each other and provide missing device information.
There are 2 lists.

List comparison: If any devices are listed below, they are in... 
 • AEG Project Implementation Roll Out Schedule.csv but are missing from... 
 • Missing from Copy of AUGUST 2019 WELLS FARGO - ANSCHUTZ ENTERTAINMENT INVOICE.csv
Coding troubleshooting (if needed) note: Iterator index: 0. 
compare_to_index: 1.

Comparison stats:
 • Quantity of primary lists' entries: 51
 • Matches found: 45
 • Total Missing: 6 
Please investigate then add applicable record(s). Some Key fields: 

----------------------------------------------------------
serial_number    ACTION         Completed Install Date    Location Name
---------------  -------------  ------------------------  -----------------

SCNCJ36223       CONVERT TRIAL  04/19/19                  Regency SF
SCNEJ51926       REPLACE        07/17/19                  AEG Legal

'\n    TO DO\n    Reverse order check e.g. 2 against 1\n    Dyanmic or earlier selected additional data fields e.g. "Install Status" vs "Status of delivery/install"\n'

## Missing Records Part 2 - Reverse Order

In [51]:
print("----------------------------------------------------------")

#TO DO: itereate among filtered lists to check against unique serials lists

total_lists = len(source_file_lists)

print("This section will compare each list against each other and provide missing device information.\nThere are {} lists.".format(total_lists))  

if total_lists > 1:
    for num, data_set in reversed(list(enumerate(source_file_unique_filtered_data, start=total_lists-2))):
        compare_to_index = num-1
        while compare_to_index >= 0:
            print(color.UNDERLINE +'\nList comparison:' + color.END + ' If any devices are listed below, they are in... \n'  + \
            " • " + "" + color.GREEN + source_file_names[num] + color.END + " but are missing from... \n" + \
            " • " +  color.BOLD + color.RED + "Missing from " + source_file_names[compare_to_index] + color.END)
            print("Coding troubleshooting (if needed) note: Iterator index: {}. \ncompare_to_index: {}.".format(num, \
                                                                        compare_to_index))    
            locals()["data_set_matches_" + str(num)] = check_and_list_missing_serials_in_lists(source_file_unique_filtered_data[num], \
                                                                                               source_file_unique_serials[compare_to_index])
            #source_file_unique_filtered_data.append(locals()["data_set_unique_filtered_data_" + str(num)])
            compare_to_index = compare_to_index-1

print("\nI did the thing!")

----------------------------------------------------------
This section will compare each list against each other and provide missing device information.
There are 2 lists.

List comparison: If any devices are listed below, they are in... 
 • Copy of AUGUST 2019 WELLS FARGO - ANSCHUTZ ENTERTAINMENT INVOICE.csv but are missing from... 
 • Missing from AEG Project Implementation Roll Out Schedule.csv
Coding troubleshooting (if needed) note: Iterator index: 1. 
compare_to_index: 0.

Comparison stats:
 • Quantity of primary lists' entries: 82
 • Matches found: 45
 • Total Missing: 37 
Please investigate then add applicable record(s). Some Key fields: 

----------------------------------------------------------
serial_number    Model           Ship-To Name                      Ship To Address1           Install Date
---------------  --------------  --------------------------------  -------------------------  --------------

S75289200F3ZHX   ESTUDIO389CS    ANSCHUTZ ENTERTAINMENT GROUP INC  

## Missing Records - Find/List Missing Serials PART 3

In [46]:

# if src1 and src3:
#     print(color.UNDERLINE +'\nList comparison:' + color.END + ' If any devices are listed below, they are in... \n'  + \
#          " • " + "" + color.GREEN + src1 + color.END + " but are... \n" + \
#          " • " +  color.BOLD + color.RED + "Missing from " + src3 + color.END)

#     ib_found_in_ps = check_and_list_missing_serials_in_lists(ib_data_filtered, ros_devices_unique_device_serials)


## Missing Records - Find/List Missing Serials PART 4

In [47]:
# if src3 and src4:
#     print(color.UNDERLINE +'\nList comparison:' + color.END + ' If any devices are listed below, they are in... \n'  + \
#          " • " + "" + color.GREEN + src3 + color.END + " but are... \n" + \
#          " • " +  color.BOLD + color.RED + "Missing from " + src4 + color.END)
# #
#     ros_found_in_billing = check_and_list_missing_serials_in_lists(ros_data_filtered_for_ps, billing_devices_unique_device_serials)


## Deeper dive into IB FUNCTION

In [48]:
#comparison function between two lists 
def found_and_missing_devices(list_a, list_b):
    """
    #####################################
        Takes in two lists (each list is a list of dicts for a devices e.g. serial_number key to asdf1234 value)
            and compares them to each other finding matches and "not_found" based on serial_number.
        Args: 
            list_a - entries to be check, return values are based on these entries
            list_b - entires to check against
        Returns:
            found_matches - a list of entries from list_a that were found in list_b based on serial_number
            not_found - a list of entries from list_a that were NOT found in list_b based on serial_number
    #####################################
    """
    #Print output 
    print(color.UNDERLINE +'\nComparison stats:\n' + color.END + \
     " • " + 'Quantity of primary lists\' entries: ' + str(len(list_a)))

    found_matches = list()
    not_found = list()
    for device in list_a:
        device_serial = device['serial_number'] 
        if check_serial_silly_s(device_serial, list_b):
            found_matches.append()
            continue
        else:
            not_found.append(device)
            
    #Print output         
    print(" • " + 'Matches found: ' + str(len(found_matches)))

    if (not_found):
        output_sample_records(not_found)

    return found_matches, not_found

print("I did the thing!")

I did the thing!


## IB DEEPER DIVE - Import full list of IB serials

In [49]:
## Deeper dive with full IB Serial list using NumPy
import pandas as pd
import numpy as np
if src_large_1 and src1: 
    full_ib_serials = np.array(pd.read_csv(src_large_1))
    print (full_ib_serials.dtype)
    print(full_ib_serials[0])


object
['SCLJ585945']


## IB DEEPER DIVE - Output list of serials found in project but not found in IB Customer nor IB Full.  TOTALLY MISSING FROM IB.

In [50]:
if src_large_1 and src1: 
    print("There were the following number NOT found in IB: ", len(ros_found_in_ib[1]))
    devices_missing_vs_found_in_full_ib = check_and_list_missing_serials_in_lists(ros_found_in_ib[1],full_ib_serials)

NameError: name 'ros_found_in_ib' is not defined

## IB DEEPER DIVE - Record accuracy issue - Found in full IB data but NOT customer-number IB

In [ ]:
print("Please investigate then correct issues for these record(s). Some Key fields: \n")

# variables: devices_missing_vs_found_in_full_ib [0=found in previous comparison, 1=not found]
# We want to list the IB details about the ones found. Total list found in devices_missing_vs_found_in_full_ib[0]
if src_large_1 and src1: 
    if (devices_missing_vs_found_in_full_ib[0]):
        output_sample_records(devices_missing_vs_found_in_full_ib[0])



## IB DEEPER DIVE - Import full IB BRIEF list to get more IB information (resource heavy operation)

In [ ]:
## Deeper dive with full IB Serial list using NumPy
if src_large_2 and src1:
    full_ib_brief_serials = np.array(pd.read_csv(src_large_2))
    print (full_ib_brief_serials.dtype)
    print(full_ib_brief_serials[0])
    print ('I did the thing!)')

## IB DEEPER DIVE: Output IB data found in project but not customer-IB. i.e. Same as above but with data from IB instead of project. (resource heavy operation)

In [ ]:
print("Please investigate then correct issues for these record(s). Some Key fields: \n")
if src_large_2 and src1: 
    if (devices_missing_vs_found_in_full_ib[0]):
        rows = list()
        #Print output         
        print (" • " + 'Total Found with issue: ' + str(len(devices_missing_vs_found_in_full_ib[0])) + " \n" \
        + color.BOLD + "Please investigate then correct applicable record(s). Some Key fields: \n" + color.END)
        print("These may not have known customer nubmers: T0BS49A, T0BUQW1, T0BV7VF, T0BVMSY")
        print("Ken's Observation on customer numbers: 506502, 507534, 640983")
    
    #Check if IB headers exist and then output
    headers = ('serial_number','INSTALL_DATE','CUSTOMER_REFERENCE','CUSTOMER_NAME', "CUSTOMER_LOCATION_ADDRESS_1")
    for entry in devices_missing_vs_found_in_full_ib[0]:
        for ib_entry in full_ib_brief_serials:
            if entry['serial_number'] == ib_entry[5]:
                rows.append([ib_entry[5], ib_entry[0], ib_entry[1],
                             ib_entry[2], ib_entry[3]])
                break

    print(tabulate(rows, headers))

## Summary of Devices Found vs. Not-Found between Various Databases

In [ ]:
print("Project Devices Installed: " + str(len(ros_devices_unique_device_serials)))

print("\n • " + "Project Devices found in the Billing file: " + str(len(ros_found_in_billing[0])) + \
"\n • " + "And those not found: " +  str(len(ros_found_in_billing[1]))) if src4 and src3 else None

print( "\n • " + "Project Devices found in PageSmart: " + str(len(ros_found_in_ps[0])) + \
"\n • " + "And those not found: " +  str(len(ros_found_in_ps[1])) ) if src2 and src3 else None


print( "\n • " + "Project Devices found in IB (Oracle Installed Base): " + str(len(ros_found_in_ib[0])) + \
"\n • " + "And those not found: " +  str(len(ros_found_in_ib[1])) ) if src1 and src3 else None

print("\n   • " + "Of which were found in IB with data issue: " + str(len(devices_missing_vs_found_in_full_ib[0])) + \
"\n   • " + "And those not found at all: " +  str(len(devices_missing_vs_found_in_full_ib[1])) ) if src_large_1 and src1 else None


print("\nInstalled Base (Oracle) Devices pulled using customer numbers: ", cust_numbers )

print("Installed Base (Oracle) Devices: " + str(len(ib_devices_unique_device_serials)) + \
    "\n • " + "IB Devices found in PageSmart: " + str(len(ib_found_in_ps[0])) + \
    "\n • " + "And those not found: " +  str(len(ib_found_in_ps[1])) ) if src1 and src2 else None


## Export Spreadsheet

In [ ]:
# with open("exported_spreadsheet.csv", 'w', newline='') as myfile:
#      wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#      wr.writerow(ros_found_in_billing[1])
    
# np.savetxt("file_name.csv", ros_found_in_billing[1], delimiter=",", fmt='%s', header=header)

# print(ros_found_in_billing[1][0].keys())

# toCSV = [ros_found_in_billing[1]]
# keys = ros_found_in_billing[1][0].keys()
# with open('people.csv', 'wb') as output_file:
#     dict_writer = newline.DictWriter(output_file, keys)
#     dict_writer.writeheader()
#     dict_writer.writerows(toCSV)

# import pandas
# dataframe = pandas.read_csv("exported_spreadsheet.csv")
# list_of_dictionaries = dataframe.to_dict(ros_found_in_billing[1])
# dataframe.to_csv("exported_spreadsheet.csv")

# with open ('list.csv', 'w') as f:
#     for dict in ros_found_in_billing[1]:
#         for key, value in dict.items():
#             text = key+','+value+'\n'
#             f.writelines(text)

# keys = [i for s in [d.keys() for d in ros_found_in_billing[1]] for i in s]

# with open('test.csv', 'a') as output_file:
#     dict_writer = csv.DictWriter(output_file, restval="-", fieldnames=keys, delimiter='@')
#     dict_writer.writeheader()
#     dict_writer.writerows(ros_found_in_billing[1])

# keys = [i for s in [d.keys() for d in ros_found_in_billing[1]] for i in s]
# f = open("sample.csv", "w")
# writer = csv.DictWriter(
#     f, fieldnames=keys)
# writer.writeheader()
# writer.writerows(ros_found_in_billing[1])
# f.close()

## next topic

## next topic

## next topic

## next topic

## next topic

## next topic

## next topic